## Гибридные рекомендательные системы

In [135]:
import pandas as pd
import numpy as np

In [136]:
links = pd.read_csv('data/ml-latest-small/links.csv')
movies = pd.read_csv('data/ml-latest-small/movies.csv')
ratings = pd.read_csv('data/ml-latest-small/ratings.csv')
tags = pd.read_csv('data/ml-latest-small/tags.csv')

In [137]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [138]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [139]:
#объединяем датасеты
movies_and_ratings =  movies.join(ratings.set_index('movieId'), on='movieId').reset_index(drop=True)
movies_and_ratings.dropna(inplace=True)

In [140]:
movies_and_ratings.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1.0,4.0,9.649827e+08
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5.0,4.0,8.474350e+08
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7.0,4.5,1.106636e+09
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15.0,2.5,1.510578e+09
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17.0,4.5,1.305696e+09


In [141]:
dataset = pd.DataFrame({
    'user_id': movies_and_ratings.userId,
    'item_id': movies_and_ratings.title,
    'rating' : movies_and_ratings.rating
})

In [142]:
from surprise import Dataset
from surprise import Reader

In [143]:
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(dataset, reader)
data

In [144]:
from surprise.model_selection import train_test_split
train, test = train_test_split(data, test_size=.15, random_state=1)

#### SVD

In [145]:
from surprise import SVD
#Каждый фактор измеряет, сколько пользователей связаны с элементом. 
#Количество факторов также является размерностью пространства скрытых факторов. 
#С увеличением числа пользователей и элементов лучше установить большее количество факторов. 
#Но если число слишком велико, производительность может быть снижена.

algoritm_svd = SVD(n_factors=20, n_epochs=20)
algoritm_svd.fit(train)
prediction = algoritm_svd.test(test)

In [146]:
from surprise import accuracy

accuracy.rmse(prediction, verbose=True)

RMSE: 0.8654


0.8654220163949454

In [147]:
#Предсказываю рейтинг который поставит пользователь фильму
algoritm_svd.predict(uid=2.0, iid='No Game No Life: Zero (2017)').est

3.4938862359336986

#### Формируем датасет из фильмов которые он не смотрел и предсказанного к ним рейтинга

In [148]:
#Соберем фильмы которые есть у пользователя 2
current_user_id = 2.0
user_movies = movies_and_ratings[movies_and_ratings.userId == current_user_id].title.unique()
#фильмы которые поьзователь 2.0 смотрел
user_movies

array(['Shawshank Redemption, The (1994)', 'Tommy Boy (1995)',
       'Good Will Hunting (1997)', 'Gladiator (2000)',
       'Kill Bill: Vol. 1 (2003)', 'Collateral (2004)',
       'Talladega Nights: The Ballad of Ricky Bobby (2006)',
       'Departed, The (2006)', 'Dark Knight, The (2008)',
       'Step Brothers (2008)', 'Inglourious Basterds (2009)',
       'Zombieland (2009)', 'Shutter Island (2010)',
       'Exit Through the Gift Shop (2010)', 'Inception (2010)',
       'Town, The (2010)', 'Inside Job (2010)',
       'Louis C.K.: Hilarious (2010)', 'Warrior (2011)',
       'Dark Knight Rises, The (2012)',
       'Girl with the Dragon Tattoo, The (2011)',
       'Django Unchained (2012)', 'Wolf of Wall Street, The (2013)',
       'Interstellar (2014)', 'Whiplash (2014)', 'The Drop (2014)',
       'Ex Machina (2015)', 'Mad Max: Fury Road (2015)',
       'The Jinx: The Life and Deaths of Robert Durst (2015)'],
      dtype=object)

In [149]:
scores = []
titles = []

for movie in movies_and_ratings.title.unique():
    if movie in user_movies:
        continue
    #предсказываем рейтинг для фильмов которые не сомтрел пользователь 2.0
    scores.append(algoritm_svd.predict(uid=current_user_id, iid=movie).est)
    titles.append(movie)

In [151]:
pd.DataFrame({'title': titles, 'scores': scores}).sort_values('scores', ascending=False).head(10)

,title,scores
600,Dr. Strangelove or: How I Learned to Stop Worr...,4.417614
46,"Usual Suspects, The (1995)",4.361373
903,Lawrence of Arabia (1962),4.348520
688,North by Northwest (1959),4.347012
898,Brazil (1985),4.335666
692,Casablanca (1942),4.329820
678,"Philadelphia Story, The (1940)",4.313280
2221,Fight Club (1999),4.312448
1730,American History X (1998),4.311866
838,"Streetcar Named Desire, A (1951)",4.297743


**Извлекаем жанры фильмов**

In [152]:
movies.genres

0       Adventure|Animation|Children|Comedy|Fantasy
1                        Adventure|Children|Fantasy
2                                    Comedy|Romance
3                              Comedy|Drama|Romance
4                                            Comedy
                           ...                     
9737                Action|Animation|Comedy|Fantasy
9738                       Animation|Comedy|Fantasy
9739                                          Drama
9740                               Action|Animation
9741                                         Comedy
Name: genres, Length: 9742, dtype: object

In [153]:
#функция очищающая посторонние символы
def change_string(s):
    return ' '.join(s.replace(' ','').replace('-','').split('|'))

In [154]:
#извлекаем жанры
movies_genres = [change_string(g) for g in movies.genres.values]

movies_genres[:2]

#жанры разделяем пробелами, т.к CountVectorizer и Transform считаю пробел разделителями 

['Adventure Animation Children Comedy Fantasy', 'Adventure Children Fantasy']

**Извлекаем 20 ближайших фильмов**

Алгоритмы машинного обучения не воспринимают слова, понимают только цифры, поэтому жанры преобразуем в числа

In [155]:
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer

In [156]:
#преобразовывает входной текст в матрицу, значением которой являются количества вхождения данного слова в текст
count_vector = CountVectorizer()#его эквивалент TfidVectorizer 

tft = TfidfTransformer()#используется для оценки важности слова

In [157]:
#преобразовываем жанры в матрицу CountVectorizer
train_counts = count_vector.fit_transform(movies_genres)

train_counts.toarray()

array([[0, 1, 1, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [158]:
#преобразуем матрицу CountVectorizer в TfidfTransformer (важность слов)
train_tft = tft.fit_transform(train_counts)

train_tft.toarray()

array([[0.        , 0.41684567, 0.51622547, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.51236121, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.57860574, 0.        , 0.81560738, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [159]:
from sklearn.neighbors import NearestNeighbors

neigh = NearestNeighbors(n_neighbors=20, n_jobs=-1, metric='euclidean')
neigh.fit(train_tft)

NearestNeighbors(metric='euclidean', n_jobs=-1, n_neighbors=20)

In [160]:
#возьмум жанры
test = change_string("Adventure|Comedy|Fantasy|Crime")
#преобразуем в веса
predict = count_vector.transform([test])
train_tft_2 = tft.transform(predict)

res = neigh.kneighbors(train_tft_2, return_distance=True)

In [161]:
#получаем ближайших соседей, id фильмов которые близки к жанрам Adventure|Comedy|Fantasy|Crime
res

(array([[0.42079615, 0.53300564, 0.54288608, 0.54288608, 0.54288608,
         0.54288608, 0.54288608, 0.54288608, 0.54288608, 0.54288608,
         0.54288608, 0.54288608, 0.54288608, 0.54288608, 0.54288608,
         0.54288608, 0.54288608, 0.6188388 , 0.62682864, 0.62682864]]),
 array([[6774, 9096, 3576,  863, 2302, 2608, 7865, 3582, 8361, 3302, 5737,
         6723, 5636, 3376, 7496, 5627, 9717, 2206, 6133, 5832]]))

In [162]:
res[1][0]

array([6774, 9096, 3576,  863, 2302, 2608, 7865, 3582, 8361, 3302, 5737,
       6723, 5636, 3376, 7496, 5627, 9717, 2206, 6133, 5832])

In [163]:
res[0][0]

array([0.42079615, 0.53300564, 0.54288608, 0.54288608, 0.54288608,
       0.54288608, 0.54288608, 0.54288608, 0.54288608, 0.54288608,
       0.54288608, 0.54288608, 0.54288608, 0.54288608, 0.54288608,
       0.54288608, 0.54288608, 0.6188388 , 0.62682864, 0.62682864])

In [164]:
#смотрим какие фильмы наиболее близки по жанрам
movies.iloc[res[1][0]]

,movieId,title,genres
6774,60074,Hancock (2008),Action|Adventure|Comedy|Crime|Fantasy
9096,143559,L.A. Slasher (2015),Comedy|Crime|Fantasy
3576,4899,Black Knight (2001),Adventure|Comedy|Fantasy
863,1136,Monty Python and the Holy Grail (1975),Adventure|Comedy|Fantasy
2302,3052,Dogma (1999),Adventure|Comedy|Fantasy
2608,3489,Hook (1991),Adventure|Comedy|Fantasy
7865,94015,Mirror Mirror (2012),Adventure|Comedy|Fantasy
3582,4911,Jabberwocky (1977),Adventure|Comedy|Fantasy
8361,109042,Knights of Badassdom (2013),Adventure|Comedy|Fantasy
3302,4467,"Adventures of Baron Munchausen, The (1988)",Adventure|Comedy|Fantasy


In [165]:
#сортировка значений по времени
movies_and_ratings.sort_values('timestamp', inplace=True)
movies_and_ratings.head()

,movieId,title,genres,userId,rating,timestamp
15993,590,Dances with Wolves (1990),Adventure|Drama|Western,429.0,5.0,828124615.0
5936,222,Circle of Friends (1995),Drama|Romance,429.0,4.0,828124615.0
12093,434,Cliffhanger (1993),Action|Adventure|Thriller,429.0,4.0,828124615.0
16167,592,Batman (1989),Action|Crime|Thriller,429.0,5.0,828124615.0
6119,225,Disclosure (1994),Drama|Thriller,429.0,4.0,828124615.0


In [166]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [167]:
#создадим словарь title - movies
title_genres = {}

for index, row in movies.iterrows():
    title_genres[row.title] = row.genres

title_genres 

{'Toy Story (1995)': 'Adventure|Animation|Children|Comedy|Fantasy',
 'Jumanji (1995)': 'Adventure|Children|Fantasy',
 'Grumpier Old Men (1995)': 'Comedy|Romance',
 'Waiting to Exhale (1995)': 'Comedy|Drama|Romance',
 'Father of the Bride Part II (1995)': 'Comedy',
 'Heat (1995)': 'Action|Crime|Thriller',
 'Sabrina (1995)': 'Comedy|Romance',
 'Tom and Huck (1995)': 'Adventure|Children',
 'Sudden Death (1995)': 'Action',
 'GoldenEye (1995)': 'Action|Adventure|Thriller',
 'American President, The (1995)': 'Comedy|Drama|Romance',
 'Dracula: Dead and Loving It (1995)': 'Comedy|Horror',
 'Balto (1995)': 'Adventure|Animation|Children',
 'Nixon (1995)': 'Drama',
 'Cutthroat Island (1995)': 'Action|Adventure|Romance',
 'Casino (1995)': 'Crime|Drama',
 'Sense and Sensibility (1995)': 'Drama|Romance',
 'Four Rooms (1995)': 'Comedy',
 'Ace Ventura: When Nature Calls (1995)': 'Comedy',
 'Money Train (1995)': 'Action|Comedy|Crime|Drama|Thriller',
 'Get Shorty (1995)': 'Comedy|Crime|Thriller',
 'Copy

## Каскадный подход

**Шаг 1. Отбор кандидатов**

In [168]:
def selection_of_candidates(user_id):
    current_user_id = user_id
    #список фильмов просмотренных пользователем
    user_movies = movies_and_ratings[movies_and_ratings.userId == current_user_id].title.unique()
    #посленйи фильм
    last_user_movie = user_movies[-1]
    #получим жанры последнего фильма
    movie_genres = title_genres[last_user_movie]
    #к примеру 'Crime|Drama|Thriller' очистим от лишних символов для дальнейшего преобразования
    movie_genres = change_string(movie_genres)
    #преобразовываем жанры в матрицу CountVectorizer
    count_matrix = count_vector.transform([movie_genres])
    #преобразовываем матрицу CountVectorizer в матрицу весов TfidfTransformer 
    tft_matrix = tft.transform(count_matrix)
    #воспользуемся методом ближайших соседей для подбора наиболее похожих жанров
    res = neigh.kneighbors(tft_matrix, return_distance=True)
    #список фильмов для оценки
    movies_to_score = movies.iloc[res[1][0]].title.values
    return movies_to_score

**~ Промежуточные вычисления начало~**

In [169]:
movie_genres

'Crime Drama Thriller'

In [170]:
# чтобы получить сгенерированный словарь, из приведенной структуры CountVectorizer, стоит отметить что порядок совпадает с матрицей
count_vector.get_feature_names()

['action',
 'adventure',
 'animation',
 'children',
 'comedy',
 'crime',
 'documentary',
 'drama',
 'fantasy',
 'filmnoir',
 'horror',
 'imax',
 'musical',
 'mystery',
 'nogenreslisted',
 'romance',
 'scifi',
 'thriller',
 'war',
 'western']

In [171]:
count_matrix.toarray()

array([[0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]])

In [172]:
res

(array([[0.42079615, 0.53300564, 0.54288608, 0.54288608, 0.54288608,
         0.54288608, 0.54288608, 0.54288608, 0.54288608, 0.54288608,
         0.54288608, 0.54288608, 0.54288608, 0.54288608, 0.54288608,
         0.54288608, 0.54288608, 0.6188388 , 0.62682864, 0.62682864]]),
 array([[6774, 9096, 3576,  863, 2302, 2608, 7865, 3582, 8361, 3302, 5737,
         6723, 5636, 3376, 7496, 5627, 9717, 2206, 6133, 5832]]))

**~ Промежуточные вычисления конец~**

**Из того, что было выбрано будем производить оценку с помощью алгоритма SVD (делать ранжирование)**

**Шаг 2. Предсказание оценки**

In [132]:
#список наиболее близких пользователю фильмов для оценки
movies_to_score=selection_of_candidates(2.0)
movies_to_score

array(['Street Kings (2008)', 'Traitor (2008)', 'I, the Jury (1982)',
       'The Drop (2014)', 'Infernal Affairs (Mou gaan dou) (2002)',
       'City of Lost Souls, The (Hyôryuu-gai) (2000)',
       'Captive, The (2014)', 'Narc (2002)', 'Last Seduction, The (1994)',
       'Simple Plan, A (1998)', 'Amateur (1994)', 'Transsiberian (2008)',
       'Thief (1981)', 'Nightcrawler (2014)', 'Town, The (2010)',
       'Badlands (1973)', 'Zulu (2013)', 'Headshot (2011)',
       'Undertow (2004)', 'Cape Fear (1962)'], dtype=object)

In [208]:
scores = []
titles = []

for movie in movies_to_score:
    if movie in user_movies:
        continue

    #предсказываем рейтинг для фильмов которые не сомтрел пользователь 2.0
    scores.append(algoritm_svd.predict(uid=current_user_id, iid=movie).est)
    titles.append(movie)

#сортируем и получаем не значения а отсортированные индексы
best_indexes = np.argsort(scores)[-10:]
#обратная сортировка
for i in reversed(best_indexes):
   print(str(titles[i]) + '  ---->  '  + str(scores[i]))

Infernal Affairs (Mou gaan dou) (2002)  ---->  3.9984703661259986
Badlands (1973)  ---->  3.9486347443045635
Nightcrawler (2014)  ---->  3.922618124177759
Simple Plan, A (1998)  ---->  3.8007272043110394
Cape Fear (1962)  ---->  3.7385973814001736
I, the Jury (1982)  ---->  3.6687864882893786
Last Seduction, The (1994)  ---->  3.6432785120559044
Thief (1981)  ---->  3.6342776458594708
Narc (2002)  ---->  3.6210717626631754
Traitor (2008)  ---->  3.58274810043244
